In [30]:
import numpy as np

locations = ['Amsterdam', 'Den Haag', 'Leiden', 'Utrecht', 'Heerlen', 'Maastricht', 
             'Groningen', 'Leeuwarden', 'Rotterdam', 'Middelburg', 'Delft', 'Almere']
timeslots = [1, 2, 3, 4]
trains = {}
np.random.seed(1)
for origin in locations:
    destinations = []
    for destin in locations:
        if destin != origin:
            train = {}
            train['Destination'] = destin
            train['TrainsHour'] = np.random.randint(1,5)
            train['TravelHour'] = (8, 23)
            train['TravelTime'] = np.random.randint(10, 240)
            destinations.append(train)
    trains[origin] = destinations
    
def trains_leaving(origin, dep_time):
    trains_leaving = []
    time = dep_time.split(':')
    hour = int(time[0])
    minutes = int(time[1])
    for train in origin:
        if hour >= train['TravelHour'][0] and hour < train['TravelHour'][1]:
            for i in range(1, train['TrainsHour']+1):
                if i*minutes == 60 or i*minutes == 0:
                    trains_leaving.append(train['Destination'])
    for destin in list(set(trains_leaving)):
        print("The train to", destin)
    return

def trains_arriving(data, origin, arr_time):
    trains_arriving = []
    time = arr_time.split(':')
    hour = int(time[0])
    minutes = int(time[1])
    for stations in data:
        for trains in data[stations]:
            if trains['Destination'] == origin:
                traveltime = trains['TravelTime']
                hour = int(time[0])
                minutes = int(time[1])
                for i in range(traveltime):
                    if minutes == 0:
                        minutes = 60
                        hour -= 1
                    minutes -= 1
                if hour >= trains['TravelHour'][0] and hour < trains['TravelHour'][1]:
                    for i in range(1, trains['TrainsHour']+1):
                        if i*minutes == 60 or i*minutes == 0:
                            trains_arriving.append(stations)
    for train in list(set(trains_arriving)):
        print("The train from", train)
    return

def train_to_leaving_at(data, origin, destination, dep_time):
    time = dep_time.split(':')
    hour = int(time[0])
    minutes = int(time[1])
    dep_minutes = 0
    if hour >= 9 and hour < 23:
        for train in data[origin]:
            if train['Destination'] == destination:
                step_minutes = 60/train['TrainsHour']
                for i in range(train['TrainsHour']):
                    if (i * step_minutes) >= minutes:
                        dep_minutes = int(i*step_minutes)
                        break
        if dep_minutes == 0 and minutes == 0:
            dep_minutes = '00'
        if dep_minutes == 0 and minutes != 0:
            dep_minutes = '00'
            hour += 1
        actual_dep_time = str(hour) + ":" + str(dep_minutes)
        print("The train from", origin, "to", destination, "leaves at", actual_dep_time+'.')
    else:
        print("There are no trains leaving", origin, "at this hour.")
    return
    
def train_to_arriving_at(data, origin, destination, arr_time):
    time = arr_time.split(':')
    hour = int(time[0])
    minutes = int(time[1])
    dep_minutes = 0
    for train in data[origin]:
        if train['Destination'] == destination:
            traveltime = train['TravelTime']
            for i in range(traveltime):
                if minutes == 0:
                    hour -= 1
                    minutes = 60
                minutes -= 1
            actual_dep_hour = hour
            actual_dep_minutes = minutes
            if actual_dep_hour >= train['TravelHour'][0] and actual_dep_hour < train['TravelHour'][1]:
                step_minutes = 60/train['TrainsHour']
                for j in range(1, train['TrainsHour']+1):
                    if (j * step_minutes) > actual_dep_minutes:
                        dep_minutes = int((j-1)*step_minutes)
                        break
                if dep_minutes == 0 and int(time[1]) == 0:
                    dep_minutes = '00'
                    hour += 1
                if dep_minutes == 0 and int(time[1]) != 0:
                    dep_minutes = '00'
                actual_dep_time = str(hour) + ":" + str(dep_minutes)
                print("The train to", destination, "that arrives at around", arr_time, "leaves", origin, "at", actual_dep_time+".")
            else:
                print("There are no trains arriving at", destination, "at this hour.")
    return

This is a very simple Python3 script for a bot; it will continue to ask for messages. It starts with simple-start.xml and then ask to "load aiml", which means the aiml files mentioned in the file simple-start.xml are loaded. 

Before you can use it, you have to install (using pip/pip3) python-aiml (this is the Python3 version). Make sure the aiml files are in the same folder as this notebook and you are using the python3 kernel for this notebook.

In [ ]:
import aiml

# Create the kernel and learn AIML files
kernel = aiml.Kernel()
kernel.learn("simple-start.xml")
kernel.respond("load aiml")

# Press CTRL-C to break this loop
while True:
    message = input("Enter your message >> ")
    if message == "quit":
        break #useful in notebook
        # exit() # useful in actual application
    elif message == "save":
        kernel.saveBrain("bot_brain.brn")
    else:
        bot_response = kernel.respond(message)
        
        if kernel.getPredicate("leavingTrains") == "True":
            origin = kernel.getPredicate("origin")
            dep_time = kernel.getPredicate("departure_time")
            trains_leaving(trains[origin], dep_time)
        if kernel.getPredicate("arrivingTrains") == "True":
            origin = kernel.getPredicate("origin")
            arr_time = kernel.getPredicate("arrival_time")
            trains_arriving(trains, origin, arr_time)
        if kernel.getPredicate("trainToLeaving") == "True":
            origin = kernel.getPredicate("origin")
            destination = kernel.getPredicate("destination")
            dep_time = kernel.getPredicate("departure_time")
            train_to_leaving_at(trains, origin, destination, dep_time)
        if kernel.getPredicate("trainToArriving") == "True":
            origin = kernel.getPredicate("origin")
            destination = kernel.getPredicate("destination")
            arr_time = kernel.getPredicate("arrival_time")
            train_to_arriving_at(trains, origin, destination, arr_time)
            
#       print("No way! That's my favorite Power Ranger!")
        
        # Do something with bot_response
        # syntactic parse
        # prosodic parse
        print(bot_response)

Loading simple-start.xml...done (0.00 seconds)
Loading basic_chat.aiml...done (0.00 seconds)
Loading MinReq.aiml...done (0.00 seconds)
Enter your message >> I want to arrive at Amsterdam at 11:02
Where are you traveling from today?
Enter your message >> Heerlen
The train to Amsterdam that arrives at around 11:02 leaves Heerlen at 8:00.



This is a very simple bot. You can add or change the AIML files to create a smarter chatbot which can act as a public transport information system. 

You are also allowed to make changes in the code above to change how the loop works, add python preprocessing of the responses, catch errors and so on. 